In [2]:
import agate

In [15]:
attendance = agate.Table.from_csv('/Users/jackcorrigan/projects/council-attendance/alderman-attendance/data/clean_data/final-attendance.csv')

In [16]:
print attendance

| column         | data_type |
| -------------- | --------- |
| committee      | Text      |
| date           | DateTime  |
| alderman       | Text      |
| status         | Boolean   |
| period         | Number    |
| total_absences | Number    |
| quarters       | Number    |
| thirds         | Number    |



In [17]:
grouped_alderman = attendance.group_by('alderman')

In [18]:
alderman_attendance = grouped_alderman.aggregate([
    ('Meetings_Attended', agate.Count('status',True)),('Total_Meetings', agate.Count())
])

In [19]:
alderman_attendance1 = alderman_attendance.where(lambda row: row['Total_Meetings'] > 20)

In [20]:
def f(row):
    return row[1]/row[2]

aa_all = alderman_attendance1.compute([
    ('Attendance %', agate.Formula(agate.Number(),f))
])

ordered_aa = aa_all.order_by('Attendance %', reverse=True)


table_alderman_attendance = ordered_aa.print_table(max_rows=100)

| alderman             | Meetings_Attended | Total_Meetings | Attendance % |
| -------------------- | ----------------- | -------------- | ------------ |
| Scott, Jr. Michael   |                89 |             95 |       0.937… |
| Reboyras, Ariel      |               446 |            481 |       0.927… |
| Mitchell, Gregory I. |                96 |            105 |       0.914… |
| Lopez, Raymond A.    |                92 |            102 |       0.902… |
| Hopkins, Brian       |                55 |             61 |       0.902… |
| Fioretti, Bob        |               272 |            303 |       0.898… |
| Cullerton, Timoth... |               292 |            327 |       0.893… |
| Sposato, Nicholas    |               268 |            306 |       0.876… |
| Ervin, Jason C.      |               311 |            361 |       0.861… |
| King, Sophia         |                31 |             36 |       0.861… |
| Napolitano, Antho... |                55 |             64 |       0.859… |

In [21]:
ordered_aa.aggregate(agate.Median('Attendance %'))

Decimal('0.7221404409922325231771485845')

In [22]:
ordered_aa.aggregate(agate.Mean('Attendance %'))

Decimal('0.7183945665069826620144684591')

In [23]:
committee_groups = attendance.group_by('committee')

In [24]:
aggregate_meetings = committee_groups.aggregate([('count', agate.Count())])

In [25]:
ordered_committees= aggregate_meetings.order_by('count', reverse = True)

In [26]:
date_group = committee_groups.group_by('date')

In [27]:
dates = date_group.aggregate([('count', agate.Count())])

In [28]:
totgroup = dates.group_by('committee')

In [29]:
totmeet = totgroup.aggregate([('meetings', agate.Count())])

In [30]:
ordmeet = totmeet.order_by('meetings', reverse=True)

In [31]:
tot_meetings = ordmeet.aggregate([('count', agate.Sum('meetings'))])

In [80]:
print tot_meetings

OrderedDict([('count', Decimal('775'))])


In [81]:
comm_att = committee_groups.aggregate([('count', agate.Count('status', False))])

In [82]:
total_absences = comm_att.order_by('count', reverse=True)

In [83]:
committee_attendance = committee_groups.aggregate([
    ('Total_present', agate.Count('status', True)),('total_expected', agate.Count())
])

In [84]:
def x(row):
    return row[1]/row[2]

ca_all = committee_attendance.compute([
    ('Attendance %', agate.Formula(agate.Number(),x))
])

ordered_ca = ca_all.order_by('Attendance %', reverse=True)

table_committee_attendance = ordered_ca.print_table(max_rows=100)

| committee            | Total_present | total_expected | Attendance % |
| -------------------- | ------------- | -------------- | ------------ |
| City Council         |         4,205 |          4,343 |       0.968… |
| Committee on Spec... |           497 |            637 |       0.780… |
| Committee on Avia... |           322 |            423 |       0.761… |
| Committee on Tran... |           802 |          1,083 |       0.741… |
| Committee on Lice... |           911 |          1,285 |       0.709… |
| Committee on Finance |         1,833 |          2,692 |       0.681… |
| Committee on Hous... |           773 |          1,165 |       0.664… |
| Committee on Budg... |         1,040 |          1,633 |       0.637… |
| Committee on Zoni... |           916 |          1,534 |       0.597… |
| Committee on Pede... |           165 |            285 |       0.579… |
| Committee on Publ... |           407 |            745 |       0.546… |
| Committee on Work... |           249 |           

In [85]:
ga_committee = grouped_alderman.group_by(lambda row: row['committee'])

In [86]:
ald_comm_totals = ga_committee.aggregate([
    ('Total_present', agate.Count('status', True)),('total_expected', agate.Count())
])

In [87]:
ald_comm_totals1 = ald_comm_totals.where(lambda row: row['total_expected'] > 10)

In [88]:
def z(row):
    return row[2]/row[3]

per_comm_attendance = ald_comm_totals.compute([
    ('Attendance %', agate.Formula(agate.Number(),z))
])

# per_comm_attendance_final = per_comm_attendance.order_by('Attendance %', reverse=True)

# per_comm_attendance_final2 = per_comm_attendance_final.order_by('alderman', reverse=False)
per_comm_attendance_final2 = per_comm_attendance.order_by('alderman', reverse=False)
ald_attendance_by_comm = per_comm_attendance_final2.print_table(max_rows=1000)
# table_ald_attendance_by_comm = per_comm_attendance_final2.print_table(max_rows=1000)

| alderman             | group                | Total_present | total_expected | Attendance % |
| -------------------- | -------------------- | ------------- | -------------- | ------------ |
| Arena, John          | City Council         |            87 |             87 |       1.000… |
| Arena, John          | Committee on Spec... |            27 |             34 |       0.794… |
| Arena, John          | Committee on Lice... |            54 |             70 |       0.771… |
| Arena, John          | Committee on Avia... |            21 |             22 |       0.955… |
| Arena, John          | Committee on Econ... |            33 |             44 |       0.750… |
| Arena, John          | Committee on Comm... |             7 |             14 |       0.500… |
| Arena, John          | Committee on Finance |            18 |             19 |       0.947… |
| Arena, John          | Committee on Educ... |             1 |              1 |       1.000… |
| Austin, Carrie M.    | Committee on Ho

In [89]:
alderman_attendance_tf = grouped_alderman.aggregate([
    ('Meetings_Attended', agate.Count('status',True)),('Total_Meetings', agate.Count())
])

In [90]:
grouped_alderman_tf = grouped_alderman.group_by(lambda row: row['quarters'], key_name='quarters')

In [91]:
alderman_attendance_tf = grouped_alderman_tf.aggregate([
    ('Meetings_Attended', agate.Count('status',True)),('Total_Meetings', agate.Count())
])

def d(row):
    return row[2]/row[3]

aa_all_tf = alderman_attendance_tf.compute([
    ('Attendance %', agate.Formula(agate.Number(),d))
])

ordered_aa_tf = aa_all_tf.order_by('quarters')

doubleordered_aa_tf = ordered_aa_tf.order_by('alderman', reverse=False)

table_ald_attendance_by_tf = doubleordered_aa_tf.print_table(max_rows=700)

| alderman             | quarters | Meetings_Attended | Total_Meetings | Attendance % |
| -------------------- | -------- | ----------------- | -------------- | ------------ |
| Arena, John          | 10       |                14 |             15 |       0.933… |
| Arena, John          | 11       |                 8 |             10 |       0.800… |
| Arena, John          | 12       |                12 |             14 |       0.857… |
| Arena, John          | 13       |                11 |             12 |       0.917… |
| Arena, John          | 14       |                11 |             12 |       0.917… |
| Arena, John          | 15       |                 8 |              8 |       1.000… |
| Arena, John          | 16       |                 9 |             13 |       0.692… |
| Arena, John          | 17       |                 7 |             10 |       0.700… |
| Arena, John          | 18       |                13 |             15 |       0.867… |
| Arena, John          | 19     

In [92]:
# Period 1: Jan. 2011 — June 2011
# Period 2: July. 2011 — Dec. 2011
# Period 3: Jan. 2012 — June 2012
# Period 4: July. 2012 — Dec. 2012
# Period 5: Jan. 2013 — June 2013
# Period 6: July. 2013 — Dec. 2013
# Period 7: Jan. 2014 — June 2014
# Period 8: July. 2014 — Dec. 2014
# Period 9: Jan. 2015 — June 2015
# Period 10: July. 2015 — Dec. 2015
# Period 11: Jan. 2016 — June 2016
# Period 12: July. 2016 — Jan. 2017

In [93]:
aa_by_period = attendance.group_by('period')

In [94]:
periodic_attendance = aa_by_period.aggregate([
    ('Meetings_Attended', agate.Count('status',True)),('Total_Meetings', agate.Count())
])

In [95]:
def f(row):
    return row[1]/row[2]

pa_all = periodic_attendance.compute([
    ('Attendance %', agate.Formula(agate.Number(),f))
])

ordered_pa = pa_all.order_by('period', reverse=False)


table_periodic_attendance = ordered_pa.print_table(max_rows=100)

| period | Meetings_Attended | Total_Meetings | Attendance % |
| ------ | ----------------- | -------------- | ------------ |
|      1 |               890 |            894 |       0.996… |
|      2 |             1,620 |          2,024 |       0.800… |
|      3 |             1,335 |          2,115 |       0.631… |
|      4 |             1,066 |          1,905 |       0.560… |
|      5 |             1,229 |          1,813 |       0.678… |
|      6 |             1,056 |          1,578 |       0.669… |
|      7 |             1,097 |          1,556 |       0.705… |
|      8 |             1,027 |          1,435 |       0.716… |
|      9 |               943 |          1,419 |       0.665… |
|     10 |               701 |          1,000 |       0.701… |
|     11 |             1,152 |          1,579 |       0.730… |
|     12 |             1,366 |          1,755 |       0.778… |


In [96]:
no_cc = attendance.where(lambda row: row['committee'] != 'City Council')

In [97]:
grouped_no_cc = no_cc.group_by('alderman')

In [98]:
no_cc_attendance = grouped_no_cc.aggregate([
    ('Meetings_Attended', agate.Count('status',True)),('Total_Meetings', agate.Count())
])

In [99]:
no_cc_attendance2 = no_cc_attendance.where(lambda row: row['Total_Meetings'] > 20)

In [100]:
def f(row):
    return row[1]/row[2]

no_cc_all = no_cc_attendance2.compute([
    ('No_CC_Attendance', agate.Formula(agate.Number(),f))
])

ordered_no_cc = no_cc_all.order_by('No_CC_Attendance', reverse=True)


table_no_cc_attendance = ordered_no_cc.print_table(max_rows=100)

| alderman             | Meetings_Attended | Total_Meetings | No_CC_Attendance |
| -------------------- | ----------------- | -------------- | ---------------- |
| Scott, Jr. Michael   |                67 |             73 |           0.918… |
| Reboyras, Ariel      |               361 |            394 |           0.916… |
| Mitchell, Gregory I. |                74 |             83 |           0.892… |
| Lopez, Raymond A.    |                70 |             80 |           0.875… |
| Cullerton, Timoth... |               229 |            262 |           0.874… |
| Fioretti, Bob        |               207 |            238 |           0.870… |
| Hopkins, Brian       |                33 |             39 |           0.846… |
| Sposato, Nicholas    |               182 |            219 |           0.831… |
| Dowell, Pat          |               265 |            324 |           0.818… |
| Ervin, Jason C.      |               224 |            274 |           0.818… |
| Quinn, Marty         |    

In [101]:
ordered_no_cc.aggregate(agate.Median('No_CC_Attendance'))

Decimal('0.6525332375134746676248652535')

In [102]:
ordered_no_cc.aggregate(agate.Mean('No_CC_Attendance'))

Decimal('0.6378600850345818440584141315')

In [103]:
no_cc_difference = ordered_no_cc.join(ordered_aa, left_key='alderman', right_key='alderman', inner=False, full_outer=False, require_match=True, columns=['Total_Meetings','Attendance %'])
no_cc_difference.print_table(max_rows=100)

| alderman             | Meetings_Attended | Total_Meetings | No_CC_Attendance | Total_Meetings2 | Attendance % |
| -------------------- | ----------------- | -------------- | ---------------- | --------------- | ------------ |
| Scott, Jr. Michael   |                67 |             73 |           0.918… |              95 |       0.937… |
| Reboyras, Ariel      |               361 |            394 |           0.916… |             481 |       0.927… |
| Mitchell, Gregory I. |                74 |             83 |           0.892… |             105 |       0.914… |
| Lopez, Raymond A.    |                70 |             80 |           0.875… |             102 |       0.902… |
| Cullerton, Timoth... |               229 |            262 |           0.874… |             327 |       0.893… |
| Fioretti, Bob        |               207 |            238 |           0.870… |             303 |       0.898… |
| Hopkins, Brian       |                33 |             39 |           0.846… |        

In [104]:
def f(row):
    return row[4] - row[2]

no_cc_diff = no_cc_difference.compute([
    ('CC_Meetings', agate.Formula(agate.Number(),f))
])

def x(row):
    return row[3] - row[5]

no_cc_diff2 = no_cc_diff.compute([
    ('% Change', agate.Formula(agate.Number(),x))
])


include_columns = ['alderman', 'CC_Meetings', '% Change']

full_no_cc_diff = no_cc_diff2.select(include_columns)

ordered_no_cc_diff = full_no_cc_diff.order_by('% Change', reverse=False)

In [105]:
ordered_no_cc_diff.print_table(max_rows=100)

| alderman             | CC_Meetings | % Change |
| -------------------- | ----------- | -------- |
| Chandler, Michael D. |          65 |  -0.170… |
| Munoz, Ricardo       |          87 |  -0.167… |
| Sadlowski Garza, ... |          22 |  -0.166… |
| Osterman, Harry      |          87 |  -0.165… |
| Maldonado, Roberto   |          87 |  -0.144… |
| O'Connor, Patrick    |          87 |  -0.140… |
| Ramirez-Rosa, Carlos |          22 |  -0.132… |
| Zalewski, Michael R. |          87 |  -0.132… |
| Villegas, Gilbert    |          22 |  -0.119… |
| Jackson, Sandi       |          28 |  -0.119… |
| Cardenas, George A.  |          87 |  -0.116… |
| Moore, Joseph        |          87 |  -0.113… |
| Sawyer, Roderick T.  |          87 |  -0.112… |
| Thomas, Latasha R.   |          65 |  -0.107… |
| Brookins, Jr., Ho... |          87 |  -0.106… |
| Thompson, JoAnn      |          61 |  -0.105… |
| Colon, Rey           |          65 |  -0.104… |
| Moreno, Proco Joe    |          87 |  -0.103… |


In [106]:
chairs = agate.Table.from_csv('/Users/jackcorrigan/projects/council-attendance/alderman-attendance/final_files/committee_chairs.csv')


In [107]:
print chairs

| column          | data_type |
| --------------- | --------- |
| Alderman        | Text      |
| Committee       | Text      |
| Position        | Text      |
| Attended        | Number    |
| Expected        | Number    |
| Attendance Rate | Number    |



In [108]:
no_cc_difference = ordered_aa.join(chairs, left_key='alderman', right_key='Alderman', inner=False, full_outer=False, require_match=False, columns=None)
no_cc_difference.print_table(max_rows=100)

| alderman             | Meetings_Attended | Total_Meetings | Attendance % | Committee            | Position   | ... |
| -------------------- | ----------------- | -------------- | ------------ | -------------------- | ---------- | --- |
| Scott, Jr. Michael   |                89 |             95 |       0.937… |                      |            | ... |
| Reboyras, Ariel      |               446 |            481 |       0.927… | Committee on Publ... | Chair      | ... |
| Mitchell, Gregory I. |                96 |            105 |       0.914… | Committee on Hous... | Vice Chair | ... |
| Lopez, Raymond A.    |                92 |            102 |       0.902… |                      |            | ... |
| Hopkins, Brian       |                55 |             61 |       0.902… |                      |            | ... |
| Fioretti, Bob        |               272 |            303 |       0.898… |                      |            | ... |
| Cullerton, Timoth... |               292 |    

In [109]:
chair_diff = no_cc_difference.where(lambda row: row['Committee'] is not None)

In [110]:
take_col = ['alderman', 'Position', 'Committee', 'Attendance Rate', 'Attendance %']

In [111]:
full_chair_diff = chair_diff.select(take_col)

def f(row):
    return row['Attendance Rate'] - row['Attendance %']

chair_diff_diff = full_chair_diff.compute([
    ('Difference', agate.Formula(agate.Number(),f))
])

diff_ordered = chair_diff_diff.order_by('Difference', reverse=True)

diff_ordered.print_table(max_rows=30)

| alderman             | Position   | Committee            | Attendance Rate | Attendance % | Difference |
| -------------------- | ---------- | -------------------- | --------------- | ------------ | ---------- |
| Brookins, Jr., Ho... | Chair      | Committee on Educ... |          1.000… |       0.506… |     0.494… |
| Cardenas, George A.  | Chair      | Committee on Heal... |          1.000… |       0.512… |     0.488… |
| Austin, Carrie M.    | Chair      | Committee on Budg... |          0.979… |       0.560… |     0.418… |
| Zalewski, Michael R. | Chair      | Committee on Avia... |          1.000… |       0.594… |     0.406… |
| Moore, Joseph        | Chair      | Committee on Hous... |          1.000… |       0.620… |     0.380… |
| Burke, Edward M.     | Chair      | Committee on Finance |          1.000… |       0.632… |     0.368… |
| O'Connor, Patrick    | Chair      | Committee on Work... |          0.962… |       0.603… |     0.358… |
| Solis, Daniel        | Chair      |

In [112]:
absences = agate.Table.from_csv('/Users/jackcorrigan/projects/council-attendance/alderman-attendance/final_files/absences_by_meeting.csv')

In [113]:
print absences

| column    | data_type |
| --------- | --------- |
| committee | Text      |
| date      | DateTime  |
| absences  | Number    |



In [114]:
committee_groups = absences.group_by('committee')

In [115]:
committee_counts = committee_groups.aggregate([('total_meetings', agate.Count()),('absences', agate.Sum('absences'))])

In [116]:
def f(row):
    return row[2]/row[1]

avg_absences = committee_counts.compute([
    ('avg_absences', agate.Formula(agate.Number(),f))
])

ordered_avg_absences = avg_absences.order_by('total_meetings', reverse=True)


ordered_avg_absences.print_table(max_rows=20)

| committee            | total_meetings | absences | avg_absences |
| -------------------- | -------------- | -------- | ------------ |
| City Council         |             87 |      138 |       1.586… |
| Committee on Zoni... |             86 |      618 |       7.186… |
| Committee on Finance |             77 |      859 |      11.156… |
| Committee on Lice... |             75 |      374 |       4.987… |
| Committee on Hous... |             68 |      392 |       5.765… |
| Committee on Tran... |             67 |      281 |       4.194… |
| Committee on Budg... |             47 |      593 |      12.617… |
| Committee on Econ... |             47 |      408 |       8.681… |
| Committee on Publ... |             39 |      338 |       8.667… |
| Committee on Spec... |             34 |      140 |       4.118… |
| Committee on Huma... |             29 |      233 |       8.034… |
| Committee on Work... |             26 |      230 |       8.846… |
| Committee on Avia... |             24 |      1

In [117]:
meetings = agate.Table.from_csv('/Users/jackcorrigan/projects/council-attendance/alderman-attendance/final_files/all_meetings.csv')

In [118]:
print meetings

| column    | data_type |
| --------- | --------- |
| committee | Text      |
| date      | DateTime  |
| status    | Text      |



In [119]:
committee_meetings = meetings.group_by('committee')

In [120]:
meetings_record = committee_meetings.aggregate([
    ('Missing Meetings', agate.Count('status','not recorded')),('Total Meetings', agate.Count())
])

In [121]:
def f(row):
    return row[1]/row[2]

record = meetings_record.compute([
    ('% Missing', agate.Formula(agate.Number(),f))
])

record_rank = record.order_by('% Missing', reverse=True)

table_record_rank = record_rank.print_table(max_rows=100)

| committee            | Missing Meetings | Total Meetings | % Missing |
| -------------------- | ---------------- | -------------- | --------- |
| Committee on Pede... |               54 |             74 |    0.730… |
| Committee on Heal... |               26 |             38 |    0.684… |
| Committee on Budg... |               83 |            133 |    0.624… |
| Committee on Comm... |               35 |             57 |    0.614… |
| Committee on Finance |               77 |            176 |    0.438… |
| Committee on Educ... |               10 |             26 |    0.385… |
| Committee on Hous... |               30 |            102 |    0.294… |
| Committee on Work... |                9 |             37 |    0.243… |
| Committee on Publ... |               10 |             49 |    0.204… |
| Committee on Avia... |                4 |             28 |    0.143… |
| Committee on Zoni... |               13 |            100 |    0.130… |
| Committee on Spec... |                5 |        

In [48]:
committees = attendance.where(lambda row: row['committee'] != 'City Council')
grouped_alderman_comm = committees.group_by('alderman')
grouped_alderman_commq = grouped_alderman_comm.group_by(lambda row: row['quarters'], key_name='quarters')


In [49]:
alderman_attendance_commq = grouped_alderman_commq.aggregate([
    ('Meetings_Attended', agate.Count('status',True)),('Total_Meetings', agate.Count())
])

def d(row):
    return row[2]/row[3]

aa_all_commq = alderman_attendance_commq.compute([
    ('Attendance %', agate.Formula(agate.Number(),d))
])

ordered_aa_commq = aa_all_commq.order_by('quarters')

doubleordered_aa_commq = ordered_aa_commq.order_by('alderman', reverse=False)

table_ald_attendance_by_commq = doubleordered_aa_commq.print_table(max_rows=700)

| alderman             | quarters | Meetings_Attended | Total_Meetings | Attendance % |
| -------------------- | -------- | ----------------- | -------------- | ------------ |
| Arena, John          | 10       |                10 |             11 |       0.909… |
| Arena, John          | 11       |                 5 |              7 |       0.714… |
| Arena, John          | 12       |                 6 |              8 |       0.750… |
| Arena, John          | 13       |                 8 |              9 |       0.889… |
| Arena, John          | 14       |                 7 |              8 |       0.875… |
| Arena, John          | 15       |                 6 |              6 |       1.000… |
| Arena, John          | 16       |                 2 |              6 |       0.333… |
| Arena, John          | 17       |                 4 |              7 |       0.571… |
| Arena, John          | 18       |                 9 |             11 |       0.818… |
| Arena, John          | 19     

In [50]:
doubleordered_aa_commq.to_csv('quarters_committees.csv')